# ReactFiberReconciler.js

In [ ]:
{
    import type {Fiber} from './ReactFiber';
    import type {FiberRoot} from './ReactFiberRoot';
    import type {RootTag} from 'shared/ReactRootTags';
    import type {
      Instance,
      TextInstance,
      Container,
      PublicInstance,
    } from './ReactFiberHostConfig';
    import {FundamentalComponent} from 'shared/ReactWorkTags';
    import type {ReactNodeList} from 'shared/ReactTypes';
    import type {ExpirationTime} from './ReactFiberExpirationTime';
    import type {
      SuspenseHydrationCallbacks,
      SuspenseState,
    } from './ReactFiberSuspenseComponent';

    import {
      findCurrentHostFiber,
      findCurrentHostFiberWithNoPortals,
    } from 'react-reconciler/reflection';
    import {get as getInstance} from 'shared/ReactInstanceMap';
    import {
      HostComponent,
      ClassComponent,
      HostRoot,
      SuspenseComponent,
    } from 'shared/ReactWorkTags';
    import getComponentName from 'shared/getComponentName';
    import invariant from 'shared/invariant';
    import warningWithoutStack from 'shared/warningWithoutStack';
    import ReactSharedInternals from 'shared/ReactSharedInternals';

    import {getPublicInstance} from './ReactFiberHostConfig';
    import {
      findCurrentUnmaskedContext,
      processChildContext,
      emptyContextObject,
      isContextProvider as isLegacyContextProvider,
    } from './ReactFiberContext';
    import {createFiberRoot} from './ReactFiberRoot';
    import {injectInternals} from './ReactFiberDevToolsHook';
    import {
      requestCurrentTime,
      computeExpirationForFiber,
      scheduleWork,
      flushRoot,
      batchedEventUpdates,
      batchedUpdates,
      unbatchedUpdates,
      flushSync,
      flushControlled,
      deferredUpdates,
      syncUpdates,
      discreteUpdates,
      flushDiscreteUpdates,
      flushPassiveEffects,
      warnIfNotScopedWithMatchingAct,
      warnIfUnmockedScheduler,
      IsThisRendererActing,
    } from './ReactFiberWorkLoop';
    import {createUpdate, enqueueUpdate} from './ReactUpdateQueue';
    import ReactFiberInstrumentation from './ReactFiberInstrumentation';
    import {
      getStackByFiberInDevAndProd,
      phase as ReactCurrentFiberPhase,
      current as ReactCurrentFiberCurrent,
    } from './ReactCurrentFiber';
    import {StrictMode} from './ReactTypeOfMode';
    import {
      Sync,
      computeInteractiveExpiration,
      computeContinuousHydrationExpiration,
    } from './ReactFiberExpirationTime';
    import {requestCurrentSuspenseConfig} from './ReactFiberSuspenseConfig';
    import {
      scheduleRefresh,
      scheduleRoot,
      setRefreshHandler,
      findHostInstancesForRefresh,
    } from './ReactFiberHotReloading';
}

import 内容

In [ ]:
{
    type OpaqueRoot = FiberRoot;
    
    //0 is PROD,1 is DEV
    // might add profile later
    type BundleType = 0 |1;
    
    type DevToolsConfig = {
        hundleType:BundleType,
        version:string,
        rendererPackageName:string,
        //mote this actually *does* depend on Fiber internal fields.
        //used by 'inspect clicked DOM element' in React DevToosl.
        findFiberByHostInstance?: (instance:Instance | TextInstance)=>Fiber,
        //used by RN in-app inspector.
        //this api is unfortunately RN-specific.
        //todo: change is to accept Fiber instead and type it properly.
        getInspectorDataForViewTag?:(tag:number)=>Object
    };
}

type 类型

In [ ]:
{
    let didWarnAboutNestedUpdates;
    let didWarnAboutFindNodeInStrictMode;
    
    if (__DEV__) {
      didWarnAboutNestedUpdates = false;
      didWarnAboutFindNodeInStrictMode = {};
    }
    
    function getContextForSubtree(
        parentComponent:?React$Component<any,any>
    ):Object {
            if(!parentComponent){
                return emptyContextObject;
            }
            
            const fiber = getInstance(parentComponent);
            const parentContext = findCurrentUnmaskedContext(fiber);
            
            if(fiber.tag = ClassComponent){
                const Component = fiber.type;
                if(isLegacyContextProvider(Component)){
                    return processChildContext(fiber,Component,parentContext);
                }
            }
            return parentContext;
        }
    
    function findHostInstance(component:Object):PublicInstance|null {
        const fiber =getInstance(component);
        if(fiber === undefined){
            if(typeof component.render === 'function'){
                invariant(false, 'unable to find node on an unmounted component.');
            }else{
                invariant(
                    false,
                    'Argument appears to not be a ReactComponet. Keys: %s',
                    Object.keys(component)
                )
            }
        }
        const hostFiber = findCurrentHostFiber(fiber);
        if(hostFiber=== null){
            return null;
        }
        return hostFiber.stateNode;
    }
    
    function findHostInstanceWithWarning(
        component:Object,
        methodName:string
    ):PublicInstance|null {
            if (__DEV__) {
                const fiber = getInstance(component);
                if (fiber === undefined) {
                  if (typeof component.render === 'function') {
                    invariant(false, 'Unable to find node on an unmounted component.');
                  } else {
                    invariant(
                      false,
                      'Argument appears to not be a ReactComponent. Keys: %s',
                      Object.keys(component),
                    );
                  }
                }
                const hostFiber = findCurrentHostFiber(fiber);
                if (hostFiber === null) {
                  return null;
                }
                if (hostFiber.mode & StrictMode) {
                  const componentName = getComponentName(fiber.type) || 'Component';
                  if (!didWarnAboutFindNodeInStrictMode[componentName]) {
                    didWarnAboutFindNodeInStrictMode[componentName] = true;
                    if (fiber.mode & StrictMode) {
                      warningWithoutStack(
                        false,
                        '%s is deprecated in StrictMode. ' +
                          '%s was passed an instance of %s which is inside StrictMode. ' +
                          'Instead, add a ref directly to the element you want to reference. ' +
                          'Learn more about using refs safely here: ' +
                          'https://fb.me/react-strict-mode-find-node%s',
                        methodName,
                        methodName,
                        componentName,
                        getStackByFiberInDevAndProd(hostFiber),
                      );
                    } else {
                      warningWithoutStack(
                        false,
                        '%s is deprecated in StrictMode. ' +
                          '%s was passed an instance of %s which renders StrictMode children. ' +
                          'Instead, add a ref directly to the element you want to reference. ' +
                          'Learn more about using refs safely here: ' +
                          'https://fb.me/react-strict-mode-find-node%s',
                        methodName,
                        methodName,
                        componentName,
                        getStackByFiberInDevAndProd(hostFiber),
                      );
                    }
                  }
                }
                return hostFiber.stateNode;
              }
            return findHostInstance(component);
        }
    
    function createContainer(
        containerInfo:Container,
        tag:RootTag,
        hydrate:boolean,
        hydrationCallbacks:null | SuspenseHydrationCallbacks
    ):OpaqueRoot {
            return createFiberRoot(containerInfo,tag,hydrate,hydrationCallbacks);
        }
    
    function updateContainer(
        element:ReactNodeList,
        container:OpaqueRoot,
        parentComponent:?React$Component<any,any>,
        callback:?Function
    ):ExpirationTime {
            const current = container.current;
            const currentTime = requestCurrentTime();
            if (__DEV__) {
                // $FlowExpectedError - jest isn't a global, and isn't recognized outside of tests
                if ('undefined' !== typeof jest) {
                  warnIfUnmockedScheduler(current);
                  warnIfNotScopedWithMatchingAct(current);
                }
              }
            const suspenseConfig = requestCurrentSuspenseConfig();
            const expirationTime = computeExpirationForFiber(
                currentTime,
                current,
                suspenseConfig
            );
            
            if (__DEV__) {
                if (ReactFiberInstrumentation.debugTool) {
                  if (current.alternate === null) {
                    ReactFiberInstrumentation.debugTool.onMountContainer(container);
                  } else if (element === null) {
                    ReactFiberInstrumentation.debugTool.onUnmountContainer(container);
                  } else {
                    ReactFiberInstrumentation.debugTool.onUpdateContainer(container);
                  }
                }
            }
            
            const context = getContextForSubtree(parentComponent);
            if(container.context === null){
                container.context = context;
            }else{
                container.pendingContext = context;
            }

            if (__DEV__) {
                if (
                  ReactCurrentFiberPhase === 'render' &&
                  ReactCurrentFiberCurrent !== null &&
                  !didWarnAboutNestedUpdates
                ) {
                  didWarnAboutNestedUpdates = true;
                  warningWithoutStack(
                    false,
                    'Render methods should be a pure function of props and state; ' +
                      'triggering nested component updates from render is not allowed. ' +
                      'If necessary, trigger nested updates in componentDidUpdate.\n\n' +
                      'Check the render method of %s.',
                    getComponentName(ReactCurrentFiberCurrent.type) || 'Unknown',
                  );
                }
            }
            
            const update = createUpdate(expirationTime,suspenseConfig);
            //caution:react devtools currently depends on this property
            //being called 'element'
            update.payload = {element};
            
            callback = callback === undefined? null:callback;
            if(callback!==null){
                warningWithoutStack(
                    typeof callback=== 'function',
                    'render(...): expected the last optional `callback`argument to be a '+
                    'function. instead received:%s.',
                    callback
                );
                update.callback = callback;
            }
            
            enqueueUpdate(current, update);
            scheduleWork(current,expirationTime);
            
            return expirationTime;            
        }
    
    function getPublicRootInstance(
        container:OpaqueRoot
    ):React$Component<any,any> | PublicInstance | null {
            const containerFiber = container.current;
            if(!containerFiber.child){
                return null;
            }
            switch(containerFiber.child.tag){
                case HostComponent:
                    return getPublicInstance(containerFiber.child.stateNode);
                default:
                    return containerFiber.child.stateNode;
            }
        }
    
    function attemptSynchronousHydration(fiber:Fiber):void {
        switch(fiber.tag){
            case HostRoot:
                let root:FiberRoot = fiber.stateNode;
                if(root.hydrate){
                    //flush the first scheduled 'update'
                    flushRoot(root,root.firstPendingTime);
                }
                break;
            case SuspenseComponent:
                flushSync(()=>scheduleWork(fiber,Sync));
                //if we're still blocked after this, we need to increase
                // the priority of any promises resolving within this
                //boundary so that the next attempt also has higher prl.
                let retryExpTime = computeInteractiveExpiration(requestCurrentTime());
                markRetryTimeIfNotHydrated(fiber,retryExpTime);
                break;
        }
    }
    
    function markRetryTimeImpl(fiber:Fiber,retryTime:ExpirationTime){
        let suspenseState:null | SuspenseState = fiber.memoizedState;
        if(suspenseState!==null && suspenseState.dehydrated!==null){
            if(suspenseState.retryTime<retryTime){
                suspenseState.retryTime = retryTime;
            }
        }
    }
    
    //increases the priority of thennables when they resolve within this boundary.
    function markRetryTimeIfNotHydrated(fiber:Fiber,retryTime:ExpirationTime){
        markRetryTimeImpl(fiber,retryTime);
        let alternate = fiber.alternate;
        if(alternate){
            markRetryTimeImpl(alternate,retryTime);
        }
    }
    
    function attemptUserBlockingHydration(fiber:Fiber):void{
        if(fiber.tag!==SuspenseComponent){
            //we ignore hostroots here because we can't increase
            //their priority and they should not suspend on I/O,
            //since you hava to wrap anything that might suspend in
            //suspense.
            return ;
        }
        let expTime = computeInteractiveExpiration(requestCurrentTime());
        scheduleWork(fiber,expTime);
        markRetryTimeIfNotHydrated(fiber,expTime);
    }
    
    function attemptContinuousHydration(fiber:Fiber):void{
        if(fiber.tag !== SuspenseComponent){
            // We ignore HostRoots here because we can't increase
            // their priority and they should not suspend on I/O,
            // since you have to wrap anything that might suspend in
            // Suspense.
            return;
        }
        let expTime = computeComtinuousHydrationExpiration(requestCurrentTime());
        scheduleWork(fiber,expTime);
        markRetryTimeIfNotHydrated(fiber,expTime);
    }
    
    function attemptHydrationAtCurrentPriority(fiber:Fiber):void {
        if(fiber.tag!==SuspenseComponent){
            //we ignore HostRoots here bacause we can't increase
            //their priority other than synchronously flush it.
            return ;
        }
        const currentTime = requestCurrentTime();
        const expTime = computeExpirationForFiber(currentTime,fiber,null);
        scheduleWork(fiber,expTime);
        markRetryTimeIfNotHydrated(fiber,expTime);
    }
    
    function findHostInstanceWithNoPoritals(
        fiber:Fiber
    ):PublicInstance | null {
            const hostFiber = findCurrentHostFiberWithNoPortals(fiber);
            if(hostFiber === null){
                return null;
            }
            if(hostFiber.tag === FundamentalComponent){
                return hostFiber.stateNode.instance;
            }
            return hostFiber.stateNode;
        }
    
    let shouldSuspendImpl = fiber=> false;
    
    function shouldSuspend(fiber:Fiber):boolean {
        return shouldSuspendImpl(fiber);
    }
    
    let overrideHookState = null;
    let overrideProps = null;
    let scheduleUpdate = null;
    let setSuspenseHandler = null;
    
    if (__DEV__) {
      const copyWithSetImpl = (
        obj: Object | Array<any>,
        path: Array<string | number>,
        idx: number,
        value: any,
      ) => {
        if (idx >= path.length) {
          return value;
        }
        const key = path[idx];
        const updated = Array.isArray(obj) ? obj.slice() : {...obj};
        // $FlowFixMe number or string is fine here
        updated[key] = copyWithSetImpl(obj[key], path, idx + 1, value);
        return updated;
      };

      const copyWithSet = (
        obj: Object | Array<any>,
        path: Array<string | number>,
        value: any,
      ): Object | Array<any> => {
        return copyWithSetImpl(obj, path, 0, value);
      };

      // Support DevTools editable values for useState and useReducer.
      overrideHookState = (
        fiber: Fiber,
        id: number,
        path: Array<string | number>,
        value: any,
      ) => {
        // For now, the "id" of stateful hooks is just the stateful hook index.
        // This may change in the future with e.g. nested hooks.
        let currentHook = fiber.memoizedState;
        while (currentHook !== null && id > 0) {
          currentHook = currentHook.next;
          id--;
        }
        if (currentHook !== null) {
          const newState = copyWithSet(currentHook.memoizedState, path, value);
          currentHook.memoizedState = newState;
          currentHook.baseState = newState;

          // We aren't actually adding an update to the queue,
          // because there is no update we can add for useReducer hooks that won't trigger an error.
          // (There's no appropriate action type for DevTools overrides.)
          // As a result though, React will see the scheduled update as a noop and bailout.
          // Shallow cloning props works as a workaround for now to bypass the bailout check.
          fiber.memoizedProps = {...fiber.memoizedProps};

          scheduleWork(fiber, Sync);
        }
      };

      // Support DevTools props for function components, forwardRef, memo, host components, etc.
      overrideProps = (fiber: Fiber, path: Array<string | number>, value: any) => {
        fiber.pendingProps = copyWithSet(fiber.memoizedProps, path, value);
        if (fiber.alternate) {
          fiber.alternate.pendingProps = fiber.pendingProps;
        }
        scheduleWork(fiber, Sync);
      };

      scheduleUpdate = (fiber: Fiber) => {
        scheduleWork(fiber, Sync);
      };

      setSuspenseHandler = (newShouldSuspendImpl: Fiber => boolean) => {
        shouldSuspendImpl = newShouldSuspendImpl;
      };
    }
    
    function injectIntoDevTools(devToolsConfig: DevToolsConfig): boolean {
      const {findFiberByHostInstance} = devToolsConfig;
      const {ReactCurrentDispatcher} = ReactSharedInternals;

      return injectInternals({
        ...devToolsConfig,
        overrideHookState,
        overrideProps,
        setSuspenseHandler,
        scheduleUpdate,
        currentDispatcherRef: ReactCurrentDispatcher,
        findHostInstanceByFiber(fiber: Fiber): Instance | TextInstance | null {
          const hostFiber = findCurrentHostFiber(fiber);
          if (hostFiber === null) {
            return null;
          }
          return hostFiber.stateNode;
        },
        findFiberByHostInstance(instance: Instance | TextInstance): Fiber | null {
          if (!findFiberByHostInstance) {
            // Might not be implemented by the renderer.
            return null;
          }
          return findFiberByHostInstance(instance);
        },
        // React Refresh
        findHostInstancesForRefresh: __DEV__ ? findHostInstancesForRefresh : null,
        scheduleRefresh: __DEV__ ? scheduleRefresh : null,
        scheduleRoot: __DEV__ ? scheduleRoot : null,
        setRefreshHandler: __DEV__ ? setRefreshHandler : null,
        // Enables DevTools to append owner stacks to error messages in DEV mode.
        getCurrentFiber: __DEV__ ? () => ReactCurrentFiberCurrent : null,
      });
    }
   
}